# Calculation 01 - Minimum energy lattice parameter
This notebook will let you calculate the minimum energy lattice parameters (equilibrium lattice constant and cohesive energy) of Nickel-based EAM and MEAM potentials. The first cell needs updating depending on the potential file you are using and the compound you want to run the calculation for.\
At the end of the notebook, there is a "fast run" which will allow you to run this notebook for all potentials and all phases in one go.

## 📌 Specify filenames and element related data
For any potential, please specify the filename of the potential file as follows:\
`potentials/filename.eam`\
`potentials/filename.eam.fs`\
`potentials/filename.eam.alloy`\
`potentials/filename.meam`

Then uncomment the respective lines, depending on which compound you want to run the calculation for.

**Note:** to call Non-LAMMPS variables within a LAMMPS command, please use an f-string. To call LAMMPS variables, instead of using an f-string, just use curly braces with a leading dollar sign - like this: `${x}`.

In [ ]:
potentialpath = "potentials/Ni1986.eam"

##### insert whichelements, crystalstructure, latticeconstant and outputfile here #####


'''
#fcc-Ni
compound = "Ni"
whichelements = "Ni"
crystalstructure = "fcc"
latticeconstant = 3.52
outputfile = "data/01_lattice-constant_Ni.dat"

#B2-NiAl
compound = "NiAl"
whichelements = "Ni Al"
crystalstructure = "bcc"
latticeconstant = 2.86
outputfile = "data/01_lattice-constant_NiAl.dat"

#L12-Ni3Al
compound = "Ni3Al"
whichelements = "Ni Al"
crystalstructure = "fcc"
latticeconstant = 3.56
outputfile = "data/01_lattice-constant_Ni3Al.dat"
'''

## ⬇️ Import PyLammps module

In [ ]:
from lammps import PyLammps
lmp = PyLammps()

lmp.clear()

## ⬇️ Add parameters for MEAM potentials
There is no need to change the following cell. It defines two parameters that are necessary for reading in MEAM potentials.\
**Please do note that the libraryelements variable has to contain the element symbols in the same order as they appear in the library file!**

In [ ]:
if "meam" in potentialpath:
    libraryfile = str(potentialpath.replace('.meam', '.library.meam'))
    
    if "NiAlCo2" in potentialpath:
        libraryelements = "Ni Al Co"
    elif "NiAl2" in potentialpath:
        libraryelements = "Al Ni"
    else:
        libraryelements = "Ni"

## ⬇️ Initialize simulation
`units` sets the unit system for this simulation. "metal" units are Angstroms for distance, eV for energy, etc.\
`dimension` is self-explanatory and can either be 2 or 3.\
`boundary` sets the boundary conditions in x-, y- and z-direction. Here, they are periodic in all three dimensions.\
`atom_style` specifies additional attributes depending on the style - "atomic" doesn't have any.

In [ ]:
lmp.units('metal')
lmp.dimension('3')
lmp.boundary('p p p')
lmp.atom_style('atomic')

## ⬇️ Create atoms
`lattice` specifies the predefined or customized lattice and the respective lattice constant.\
`region` specifies the simulation cell as a geometric shape, like a sphere, cylinder or block. The default coordinate unit is a "lattice parameter", meaning each coordinate is an integer factor times the lattice parameter specified in the `lattice` command. "cell" is the user-assigned ID for this specific region, but could also be called "1", "a" or anything else. For periodic boundary conditions, the cell dimensions should be multiples of the lattice constant to prevent unwanted overlaps (as explained in the documentation for the `create_atoms` command).\
`create_box` actually creates the simulation cell as defined in the `region` command. The integer defines the number of element types that will be used, the string defines the region in which the cell will be created.\
`create_atoms` adds atoms to the lattice within a specific region ("box" fills the entire simulation cell defined in the `region` command with atoms), so this has to happen after defining and creating the simulation box. The integer defines the number of element types used. "basis" allows to assign coordinates to atom types: the first integer is the coordinate (they can be found in the documentation for the `lattice` command), the second one is the element index in the same order as it will be specified when reading in the potentials (Ni = 1, Al = 2).
`replicate` would allow to change the size of the simulation, and in this position is equivalent to changing the coordinates in the `region` command.
`mass` does not need to specified for any EAM or MEAM potential since it is used from the potential file itself.

In [ ]:
lmp.lattice(f'{crystalstructure} {latticeconstant}')
lmp.region('cell block 0 1 0 1 0 1')
        
if compound == "Ni":
    lmp.create_box('1 cell')
    lmp.create_atoms('1 box')
elif compound == "NiAl":
        lmp.create_box('2 cell')
        lmp.create_atoms('2 box basis 1 1 basis 2 2')
elif compound == "Ni3Al":
        lmp.create_box('2 cell')
        lmp.create_atoms('2 box basis 1 2 basis 2 1 basis 3 1 basis 4 1')

## ⬇️ Read in potential file
`pair_style` specifies what kind of interatomic potential will be used.\
`pair_coeff` specifies the potential file to be read in. The input parameters change depending on the potential style. that the pair potential coefficients are stored in. The first two integers define the force field coefficients between atom pairs. The asterisks include all atom types.\
`neighbor` and `neigh_modify` would set parameters for the neighbor lists - the default parameters when the commands are not used can be found in the LAMMPS documentation.

In [ ]:
if "meam" in potentialpath:
    lmp.pair_style('meam')
    lmp.pair_coeff(f'* * {libraryfile} {libraryelements} {potentialpath} {whichelements}')
elif "eam.alloy" in potentialpath:
    lmp.pair_style('eam/alloy')
    lmp.pair_coeff(f'* * {potentialpath} {whichelements}')
elif "eam.fs" in potentialpath:
    lmp.pair_style('eam/fs')
    lmp.pair_coeff(f'* * {potentialpath} {whichelements}')
else:
    lmp.pair_style('eam')
    lmp.pair_coeff(f'* * {potentialpath}')

## ⬇️ Define computes
`compute`s define "formulas" that can later on be computed at each timestep of e.g. a thermodynamic run, minimization or dump output.\
The first one allows to store the potential energy for each atom during any "run".\
The second one uses the first one, summing up the potential energies of all atoms in the system.\
The third one allows to calculate the centro-symmetry parameter.

In [ ]:
lmp.compute('Epot all pe/atom')
lmp.compute('Epotall all reduce sum c_Epot')
lmp.compute('Csym all centro/atom fcc')

## ⬇️ Define thermo settings
`min_style` would set which minimization algorithm should be used for the minimization. The default value is `cg` - conjugate gradient.\
`thermo` sets the timestep interval at which computes shall be performed and printed during the run.\
`thermo_style` allows to specify what information shall be printed at each of those timesteps - in this case, it is customized to calculate and print the timestep, the length of the simulation box in x, y and z direction, and the total potential energy of the system.

In [ ]:
lmp.thermo(10)
lmp.thermo_style('custom step lx ly lz c_Epotall')

## ⬇️ Define dump file
`dump` will print the defined properties in a dump file at the desired timestep interval. With these files, the simulation can be visualized using software like OVITO.

In [ ]:
lmp.dump(f'dump all cfg 1000 dump/01_{compound}.*.cfg mass type xs ys zs c_Csym c_Epot')
lmp.dump_modify(f'dump element {whichelements}')

## ⬇️ Define fix
`fix` applies specific operations to a specified group of atoms (in this case, `all` of them). Here, the operation is to relax the entire simulation cell (`box/relax`), using the parameters `iso 0.0` (zero pressure in all directions) and `vmax 0.001` (volume change of the simulation box smaller than 0.1 % during each step).

In [ ]:
lmp.fix('1 all box/relax iso 0.0 vmax 0.001')

## ⬇️ Run minimization
`minimize`s the energy of the simulation cell, using the parameters defined in `fix(1)` and printing out the desired information (as specified in `thermo_style`) at every `thermo(10)`th timestep.

In [ ]:
lmp.minimize('1e-10 1e-10 10000 100000')

## ⬇️ Define variable formulas
This section defines some `variable`s by assigning a variable to a formula that is not yet evaluated. The first one is the cohesive energy E<sub>coh</sub>, which is the total potential energy E<sub>pot</sub> divided by the total number of atoms N, the second one is the length of the simulation cell (=equilibrium lattice constant) a.\
They will be calculated in the `print` command with the current variable values.

In [ ]:
lmp.variable('Ecoh equal c_Epotall/count(all)')
lmp.variable('latticeparameter equal lx')

## ⬇️ Undump
Close dump file.

In [ ]:
lmp.undump('dump')

## ⬇️ Print desired output
This section `print`s these values to the output file, for which you chose a filename in the beginning. It prints the filename, the cohesive energy and the lattice parameter of the chosen potential.

In [ ]:
potentialname = potentialpath.replace('potentials/', '')

lmp.print(f'"{potentialname}'+', ${Ecoh}, ${latticeparameter}" append '+f'{outputfile} screen no')

🏁**ALL DONE!**🏁

In [ ]:
def latticeparameter(compound, whichelements, crystalstructure, latticeconstant, outputfile, potentialpath):
    lmp.clear()

    if "meam" in potentialpath:
        librarypath = str(potentialpath.replace('.meam', '.library.meam'))

    if "NiAlCo2" in potentialpath:
        libraryelements = "Ni Al Co"
    elif "NiAl2" in potentialpath:
        libraryelements = "Al Ni"
    else:
        libraryelements = "Ni"

    lmp.units('metal')
    lmp.dimension('3')
    lmp.boundary('p p p')
    lmp.atom_style('atomic')

    lmp.lattice(f'{crystalstructure} {latticeconstant}')
    lmp.region('cell block 0 1 0 1 0 1')

    if compound == "Ni":
        lmp.create_box('1 cell')
        lmp.create_atoms('1 box')
    elif compound == "NiAl":
            lmp.create_box('2 cell')
            lmp.create_atoms('2 box basis 1 1 basis 2 2')
    elif compound == "Ni3Al":
            lmp.create_box('2 cell')
            lmp.create_atoms('2 box basis 1 2 basis 2 1 basis 3 1 basis 4 1')
    
    if "meam" in potentialpath:
        lmp.pair_style('meam')
        lmp.pair_coeff(f'* * {librarypath} {libraryelements} {potentialpath} {whichelements}')
    elif "eam.alloy" in potentialpath:
        lmp.pair_style('eam/alloy')
        lmp.pair_coeff(f'* * {potentialpath} {whichelements}')
    elif "eam.fs" in potentialpath:
        lmp.pair_style('eam/fs')
        lmp.pair_coeff(f'* * {potentialpath} {whichelements}')
    else:
        lmp.pair_style('eam')
        lmp.pair_coeff(f'* * {potentialpath}')

    lmp.compute('Epot all pe/atom')
    lmp.compute('Epotall all reduce sum c_Epot')

    lmp.reset_timestep(0)

    lmp.min_style('cg')

    lmp.thermo(10)
    lmp.thermo_style('custom step lx ly lz c_Epotall')

    lmp.fix('1 all box/relax iso 0.0 vmax 0.001')

    lmp.minimize('1e-10 1e-10 10000 100000')

    lmp.variable('Ecoh equal c_Epotall/count(all)')
    lmp.variable('latticeparameter equal lx')

In [ ]:
all_files = [
    "Ni1986.eam",
    "Ni1989.eam",
    "Ni1987.eam.fs",
    "Ni2012.eam.fs",
    "Ni1999.eam.alloy",
    "Ni2004.eam.alloy",
    "Ni2016.eam.alloy",
    "NiAl2002.eam.alloy",
    "NiAl2004.eam.alloy",
    "NiAl2009.eam.alloy",
    "NiAlCo2015.eam.alloy",
    "NiAlCoCrFe2020.eam.alloy",
    "Ni2003.meam",
    "Ni2015.meam",
    "Ni2018.meam",
    "NiAl2007.meam",
    "NiAl2022.meam",
    "NiAlCo2015.meam"
]

alloy_files = [
    "NiAl2002.eam.alloy",
    "NiAl2004.eam.alloy",
    "NiAl2009.eam.alloy",
    "NiAlCo2015.eam.alloy",
    "NiAlCoCrFe2020.eam.alloy",
    "NiAl2007.meam",
    "NiAl2022.meam",
    "NiAlCo2015.meam"
]

In [ ]:
compound =  ["Ni", "NiAl", "Ni3Al"]
whichelements = ["Ni", "Ni Al", "Ni Al"]
crystalstructure = ["fcc", "bcc", "fcc"]
latticeconstant =  [3.52, 2.86, 3.56]
outputfile = ["data/01_lattice-constant_Ni.dat", "data/01_lattice-constant_NiAl.dat", "data/01_lattice-constant_Ni3Al.dat"]
filelist =  [all_files, alloy_files, alloy_files]

for comp, element, crystal, lc, outputfile, files in zip (compound, whichelements, crystalstructure, latticeconstant, outputfile, filelist):
    
    for file in files:
        potentialpath = "potentials/"+file
        
        from lammps import PyLammps
        lmp = PyLammps()

        latticeparameter(comp, element, crystal, lc, outputfile, potentialpath)
        
        potentialname = potentialpath.replace('potentials/', '')
        lmp.print(f'"{potentialname}'+', ${Ecoh}, ${latticeparameter}" append '+f'{outputfile} screen no')

🏁**FAST RUN DONE!**🏁